In [14]:
%display latex

a

In [2]:
import numpy as np
s1 = matrix(
    [
        [0, 1],
        [1, 0]
    ]
)
s2 = matrix(
    [
        [0, -i],
        [i, 0]
    ]
)
s3 = matrix(
    [
        [1, 0],
        [0, -1]
    ]
)

one = matrix.identity(s1.dimensions()[0])

s = [s1, s2, s3]
s

[
[0 1]  [ 0 -I]  [ 1  0]
[1 0], [ I  0], [ 0 -1]
]

In [3]:
s1*s2 - s2*s1

[ 2*I    0]
[   0 -2*I]

In [4]:
# Hack
def Id(self, x): return self(x)

p = vector([
    function("pi"+str(i), latex_name="\\pi_"+str(i), conjugate_func=Id)(x) for i  in range(1, 4)
])
p

(pi1(x), pi2(x), pi3(x))

In [5]:
var("e", latex_name="\\varepsilon", domain="real")
var("d", latex_name="\\delta", domain="real")
var("mu", latex_name="\\mu_I", domain="real")
var("a", latex_name="\\alpha", domain="real")

pi_s = e * sum([s[i]*p[i] for i in range(len(s))])
v = 1/2*d*mu*s3

n = 5 # expand up to, not incluing, n

In [6]:
def mat_series(mat, x, n):
    d = mat.dimensions()
    for i in range(0, d[0]):
        for j in range(0, d[1]):
            mat[i, j] = mat[i, j].series(x, n).truncate()

def mat_simp(mat):
    d = mat.dimensions()
    for i in range(0, d[0]):
        for j in range(0, d[1]):
            mat[i, j] = mat[i, j].full_simplify()
            mat[i, j] = mat[i, j].trig_reduce()

def mat_prep(mat, n=n):
    mat_series(mat, e, n)
    mat_simp(mat)
    return mat

def print_e(elem):
    coeff = elem.coefficients(e)
    for i in range(len(coeff)):
        print(e^coeff[i][1], ":")
        c = coeff[i][0].coefficients(mu)
        for k in c:
            s = k[0].full_simplify()*mu**k[1]
            pretty_print(s.factor())

In [7]:
acom = lambda A1, A2 : A1 * A2 + A2 * A1
com = lambda A1, A2 : A1 * A2 - A2 * A1

def proj(A):
    A1 = A.trace()/2
    A = A - one * A1
    v = [A1, acom(A, s1)[0, 0]/2, acom(A, s2)[0, 0]/2, acom(A, s3)[0, 0]/2]
    v = [a.full_simplify().trig_reduce() for a in v]
    return v 

In [8]:
POW = lambda A, n : matrix.identity(A.dimensions()[0]) if (n == 0) else A * POW(A, n-1)
EXP = lambda A, n : sum([POW(A, i)/factorial(i) for i in range(n+1)])

In [9]:
m = Matrix([[0, 1], [-1, 0]])
POW(m, 3)

N(EXP(-pi*m, 100)) # Matrix rep. of i. Should be -one

[    -1.00000000000000 -3.46629141516451e-16]
[ 3.46629141516451e-16     -1.00000000000000]

In [10]:
A_a = one*cos(a/2) + I * s1 * sin(a/2)
U = lambda n: EXP(I * pi_s/2, n)
SIGMA = lambda n: mat_prep(A_a * U(n) * U(n) * A_a)
S = mat_prep(SIGMA(n), n=n)

$\Sigma$, to order $(\pi/f)^4$.

In [11]:
k = proj(S)
for i, ki in enumerate(k):
    print("tau_{}:".format(i))
    c = ki.coefficients(e)
    for ci in c:
        pretty_print(ci[0].full_simplify().factor())

tau_0:


cos(a)

-pi1(x)*sin(a)

-1/2*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*cos(a)

1/6*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi1(x)*sin(a)

...

I*pi2(x)

-1/6*I*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi2(x)

tau_3:


I*pi3(x)

-1/6*I*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi3(x)

In [13]:
dS = diff(S, x) # nabla_mu Sigma
dSct = diff(S.C.T, x)
COM = v*S - S*v # [v_mu, Sigma]

using $[v_\mu, \Sigma]^\dagger = - [v_\mu, \Sigma^\dagger] $

In [14]:
term1 = mat_prep(dS*(dSct))
term2 = -I*mat_prep(dS*(-COM.C.T) + COM*dSct)
term3 = mat_prep(COM*COM.C.T)
term4 = mat_prep(S + S.C.T)
terms_full = [term1, term2, term3, term4]

In [15]:
terms = [t.trace().full_simplify() for t in terms_full]

for i, t in enumerate(terms):
    print("term {}:".format(i+1))
    print_e(t)

term 1:
e^2 :


2*diff(pi1(x), x)^2 + 2*diff(pi2(x), x)^2 + 2*diff(pi3(x), x)^2

e^4 :


-2/3*pi2(x)^2*diff(pi1(x), x)^2 - 2/3*pi3(x)^2*diff(pi1(x), x)^2 + 4/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 2/3*pi1(x)^2*diff(pi2(x), x)^2 - 2/3*pi3(x)^2*diff(pi2(x), x)^2 + 4/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 4/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 2/3*pi1(x)^2*diff(pi3(x), x)^2 - 2/3*pi2(x)^2*diff(pi3(x), x)^2

term 2:
e :


...

-2*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*cos(a)

e^3 :


2/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi1(x)*sin(a)

e^4 :


1/6*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)^2*cos(a)

In [16]:
var("m", domain="real")
var("f", domain="real")

L_LO = 1/4*(term1+term2+term3+m^2*term4).trace().full_simplify().trig_reduce()
L_coeff = L_LO.coefficients(e)
L_coeff = [c[0].full_simplify().trig_reduce() for c in L_coeff]

# Leading order Lagrangian
$\mathcal{L}_4$ to $\mathcal{O}(\varepsilon^4)$, sorted by powers of $\varepsilon$ and $\mu_I$.

In [18]:
print_e(L_LO)

1 :


m^2*cos(a)

1/2*d^2*mu^2*sin(a)^2

e :


-m^2*pi1(x)*sin(a)

...

-2/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*d^2*mu^2*cos(a)*pi1(x)*sin(a)

e^4 :


1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

1/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*(pi2(x)*diff(pi1(x), x) - pi1(x)*diff(pi2(x), x))*d*mu*cos(a)

1/6*(2*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - pi1(x)^2 - pi2(x)^2)*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*d^2*mu^2

In [19]:
L_coeff[4].coefficients(mu)[0][0].trig_reduce().factor()

1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

In [20]:
(((p*p)**2*cos(a)/4*m**2 - ((p*p)*(diff(p, x)*diff(p, x) ) - (diff(p, x)*p)**2))/6).expand()

1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

### Next to leading order terms
Expanded to $\mathcal O (\varepsilon^3)$

In [21]:
S = SIGMA(3)
mat_prep(S, n=3)
[pretty_print(t.factor()) for t in proj(S)];

-1/2*e^2*cos(a)*pi1(x)^2 - 1/2*e^2*cos(a)*pi2(x)^2 - 1/2*e^2*cos(a)*pi3(x)^2 - e*pi1(x)*sin(a) + cos(a)

-1/2*I*e^2*pi1(x)^2*sin(a) - 1/2*I*e^2*pi2(x)^2*sin(a) - 1/2*I*e^2*pi3(x)^2*sin(a) + I*e*cos(a)*pi1(x) + I*sin(a)

I*e*pi2(x)

I*e*pi3(x)

In [23]:
var("xmu", latex_name="x_\\mu")
var("xnu", latex_name="x_\\nu")

var("dm", latex_name="\\Delta m", domain="real")
var("mm", latex_name="\\bar m", domain="real")
var("B0", latex_name="B_0", domain="real")

chi = 2*B0 * (mm * one + dm * s3)

$\chi \Sigma^\dagger -\Sigma \chi^\dagger$

In [24]:
chiSigmaMin = mat_prep(chi*S.C.T - S*chi.C.T, n=3)
proj(chiSigmaMin)

[-4*I*B0*dm*e*pi3(x),
 2*I*B0*e^2*mm*pi1(x)^2*sin(a) + 2*I*B0*e^2*mm*pi2(x)^2*sin(a) + 2*I*B0*e^2*mm*pi3(x)^2*sin(a) - 4*I*B0*e*mm*cos(a)*pi1(x) - 4*I*B0*mm*sin(a),
 -4*I*B0*e*mm*pi2(x),
 -4*I*B0*e*mm*pi3(x)]

In [25]:
res = (mat_prep(chiSigmaMin, n=3).trace()**2).series(e, 3).truncate()
print_e(res)

e^2 :


-64*B0^2*dm^2*pi3(x)^2

$\chi \Sigma^\dagger + \Sigma \chi^\dagger$

In [26]:
chiSigmaPlus = mat_prep(chi*S.C.T + S*chi.T.C, n=3)
[pretty_print(t.factor()) for t in proj(chiSigmaPlus)];
chiSigmaPlus.trace()

-2*(e^2*cos(a)*pi1(x)^2 + e^2*cos(a)*pi2(x)^2 + e^2*cos(a)*pi3(x)^2 + 2*e*pi1(x)*sin(a) - 2*cos(a))*B0*mm

-4*B0*dm*e*pi2(x)

-2*(e^2*pi1(x)^2*sin(a) + e^2*pi2(x)^2*sin(a) + e^2*pi3(x)^2*sin(a) - 2*e*cos(a)*pi1(x) - 2*sin(a))*B0*dm

-2*(e^2*cos(a)*pi1(x)^2 + e^2*cos(a)*pi2(x)^2 + e^2*cos(a)*pi3(x)^2 + 2*e*pi1(x)*sin(a) - 2*cos(a))*B0*dm

-4*B0*e^2*mm*cos(a)*pi1(x)^2 - 4*B0*e^2*mm*cos(a)*pi2(x)^2 - 4*B0*e^2*mm*cos(a)*pi3(x)^2 - 8*B0*e*mm*pi1(x)*sin(a) + 8*B0*mm*cos(a)

### Kinetic terms

$\nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger$

In [27]:
S2 = mat_prep(SIGMA(3), n=3)
S2ct = S2.C.T
dS = diff(S2, x)

COM = v*S2 - S2*v
term1 = mat_prep(dS*(dSct))
term2 = -I*mat_prep(COM*dSct - dS*(COM.C.T))
term3 = mat_prep(COM*COM.C.T)

In [28]:
print_e(term1.trace().series(e, 3).truncate().full_simplify().factor())
print_e(term2.trace().series(e, 3).truncate().full_simplify().factor())
print_e(term3.trace().series(e, 3).truncate().full_simplify().factor())

kin = term1 + term2 + term3

e^2 :


2*diff(pi1(x), x)^2 + 2*diff(pi2(x), x)^2 + 2*diff(pi3(x), x)^2

e :


4*d*mu*sin(a)*diff(pi2(x), x)

e^2 :


...

2*d^2*mu^2*sin(a)^2

e :


4*d^2*mu^2*cos(a)*pi1(x)*sin(a)

e^2 :


2*(2*cos(a)^2*pi1(x)^2 + cos(a)^2*pi2(x)^2 + cos(a)^2*pi3(x)^2 - pi1(x)^2 - pi3(x)^2)*d^2*mu^2

Hack needed to get answer with free indices

In [29]:
dp = diff(p, x); dp
dpmu = vector([
    function(
        "pimu"+str(i), 
        latex_name="\\partial_\\mu \\pi_"+str(i), 
        conjugate_func=Id)(x) for i  in range(1, 4)
])

dpnu = vector([
    function(
        "pinu"+str(i), 
        latex_name="\\partial_\\nu \\pi_"+str(i), 
        conjugate_func=Id)(x) for i  in range(1, 4)
])

In [30]:
dSct = diff(S2ct, x)

dSmu = copy(dS)
dSnuct = copy(dSct)
for i, dpi in enumerate(dp):
    dSmu = dSmu.subs(dpi==dpmu[i])
    dSnuct = dSnuct.subs(dpi==dpnu[i])

In [31]:
var("dmu", latex_name="\\delta_\\mu", domain="real")
var("dnu", latex_name="\\delta_\\nu", domain="real")
vmu = 1/2*dmu*mu*s3
vnu = 1/2*dnu*mu*s3

COMmu = -(vmu*S2 - S2*vmu)      # [v_mu, Sigma]
COMnu = vnu*S2ct - S2ct*vnu  # [v_nu, Sigma^\dagger]

In [32]:
term1 = mat_prep(dSmu*(dSnuct), n=3)
term2 = -I*mat_prep(dSmu*COMnu - COMmu*dSnuct, n=3)
term3 = mat_prep(COMmu*COMnu, n=3)

In [33]:
print_e(term1.trace().series(e, 3).truncate().full_simplify().factor())
print_e(term2.trace().series(e, 3).truncate().full_simplify().factor())
print_e(term3.trace().series(e, 3).truncate().full_simplify().factor())

e^2 :


2*pimu1(x)*pinu1(x) + 2*pimu2(x)*pinu2(x) + 2*pimu3(x)*pinu3(x)

e :


2*(dnu*pimu2(x) + dmu*pinu2(x))*mu*sin(a)

e^2 :


...

2*dmu*dnu*mu^2*sin(a)^2

e :


4*dmu*dnu*mu^2*cos(a)*pi1(x)*sin(a)

e^2 :


2*(2*cos(a)^2*pi1(x)^2 + cos(a)^2*pi2(x)^2 + cos(a)^2*pi3(x)^2 - pi1(x)^2 - pi3(x)^2)*dmu*dnu*mu^2

In [34]:
kin2 = term1 + term2 + term3
t = kin2.trace().series(e, 3).truncate()
print_e(t)

1 :


2*dmu*dnu*mu^2*sin(a)^2

e :


2*(dnu*pimu2(x) + dmu*pinu2(x))*mu*sin(a)

4*dmu*dnu*mu^2*cos(a)*pi1(x)*sin(a)

e^2 :


2*pimu1(x)*pinu1(x) + 2*pimu2(x)*pinu2(x) + 2*pimu3(x)*pinu3(x)

-2*(dnu*pi2(x)*pimu1(x) - dnu*pi1(x)*pimu2(x) + dmu*pi2(x)*pinu1(x) - dmu*pi1(x)*pinu2(x))*mu*cos(a)

2*(2*cos(a)^2*pi1(x)^2 + cos(a)^2*pi2(x)^2 + cos(a)^2*pi3(x)^2 - pi1(x)^2 - pi3(x)^2)*dmu*dnu*mu^2

# Constitutive terms for NLO Lagrangian

term_l1:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla^\mu \Sigma^\dagger \}^2$

In [35]:
term_l1 = (kin.trace()**2).series(e, 3).truncate()
print_e(term_l1)

1 :


4*d^4*mu^4*sin(a)^4

e :


16*d^3*mu^3*sin(a)^3*diff(pi2(x), x)

16*d^4*mu^4*cos(a)*pi1(x)*sin(a)^3

e^2 :


8*(diff(pi1(x), x)^2 + 3*diff(pi2(x), x)^2 + diff(pi3(x), x)^2)*d^2*mu^2*sin(a)^2

-16*(pi2(x)*diff(pi1(x), x) - 3*pi1(x)*diff(pi2(x), x))*d^3*mu^3*cos(a)*sin(a)^2

-8*(4*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - 3*pi1(x)^2 - pi2(x)^2)*d^4*mu^4*sin(a)^2

term_l2:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \nabla^\mu \Sigma \nabla^\nu \Sigma^\dagger \}$

In [36]:
term_l2 = (kin2.trace()**2).series(e, 3).truncate()
print_e(term_l2)

1 :


4*dmu^2*dnu^2*mu^4*sin(a)^4

e :


8*(dnu*pimu2(x) + dmu*pinu2(x))*dmu*dnu*mu^3*sin(a)^3

16*dmu^2*dnu^2*mu^4*cos(a)*pi1(x)*sin(a)^3

e^2 :


4*(dnu^2*pimu2(x)^2 + 2*dmu*dnu*pimu1(x)*pinu1(x) + 4*dmu*dnu*pimu2(x)*pinu2(x) + dmu^2*pinu2(x)^2 + 2*dmu*dnu*pimu3(x)*pinu3(x))*mu^2*sin(a)^2

-8*(dnu*pi2(x)*pimu1(x) - 3*dnu*pi1(x)*pimu2(x) + dmu*pi2(x)*pinu1(x) - 3*dmu*pi1(x)*pinu2(x))*dmu*dnu*mu^3*cos(a)*sin(a)^2

-8*(4*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - 3*pi1(x)^2 - pi2(x)^2)*dmu^2*dnu^2*mu^4*sin(a)^2

term_l3

$\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}^2$

In [37]:
term_l3 = (chiSigmaPlus.trace()**2).series(e, 3).truncate().factor()
print_e(term_l3)

1 :


64*B0^2*mm^2*cos(a)^2

e :


-128*B0^2*mm^2*cos(a)*pi1(x)*sin(a)

e^2 :


64*(2*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - pi1(x)^2 - pi2(x)^2 - pi3(x)^2)*B0^2*mm^2

term_l4

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}$

In [38]:
term_l4 = (kin.trace()*chiSigmaPlus.trace()).series(e, 3).truncate()
print_e(term_l4)

1 :


16*B0*d^2*mm*mu^2*cos(a)*sin(a)^2

e :


32*B0*d*mm*mu*cos(a)*sin(a)*diff(pi2(x), x)

16*(3*cos(a)^2 - 1)*B0*d^2*mm*mu^2*pi1(x)*sin(a)

e^2 :


16*(diff(pi1(x), x)^2 + diff(pi2(x), x)^2 + diff(pi3(x), x)^2)*B0*mm*cos(a)

-32*(cos(a)^2*pi2(x)*diff(pi1(x), x) - 2*cos(a)^2*pi1(x)*diff(pi2(x), x) + pi1(x)*diff(pi2(x), x))*B0*d*mm*mu

-8*(9*pi1(x)^2*sin(a)^2 + 3*pi2(x)^2*sin(a)^2 + 3*pi3(x)^2*sin(a)^2 - 2*pi1(x)^2 - 2*pi2(x)^2)*B0*d^2*mm*mu^2*cos(a)

term_l7

$\mathrm{Tr}\{ \chi \Sigma^\dagger - \Sigma \chi^\dagger \}^2$

In [39]:
term_l7 = (mat_prep(chiSigmaMin, n=3).trace()**2).series(e, 3).truncate()
print_e(term_l7)

e^2 :


-64*B0^2*dm^2*pi3(x)^2

term chisq

$\mathrm{Tr}\{\chi^\dagger \chi\}$

In [40]:
term_chisq = (chi*chi).trace().series(e, 3).truncate()
print_e(term_chisq)

1 :


8*(dm^2 + mm^2)*B0^2

last_term

$\mathrm{Tr}\{ (\chi \Sigma^\dagger)^2 + (\Sigma \chi^\dagger)^2 \}$

In [41]:
term_last = ((chi*S2ct)**2 + (S2*chi)**2).trace().series(e, 3)
print_e(term_last)

1 :


-16*(2*mm^2*sin(a)^2 - dm^2 - mm^2)*B0^2

e :


-64*B0^2*mm^2*cos(a)*pi1(x)*sin(a)

e^2 :


32*(2*mm^2*pi1(x)^2*sin(a)^2 + mm^2*pi2(x)^2*sin(a)^2 + mm^2*pi3(x)^2*sin(a)^2 - mm^2*pi1(x)^2 - mm^2*pi2(x)^2 - dm^2*pi3(x)^2 - mm^2*pi3(x)^2)*B0^2

term_l1:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla^\mu \Sigma^\dagger \}^2$

term_l2:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \nabla^\mu \Sigma \nabla^\nu \Sigma^\dagger \}$

term_l3

$\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}^2$

term_l4

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}$

term_l7

$\mathrm{Tr}\{ \chi \Sigma^\dagger - \Sigma \chi^\dagger \}^2$

term chisq

$\mathrm{Tr}\{\chi^\dagger \chi\}$

term_last

$\mathrm{Tr}\{ (\chi \Sigma^\dagger)^2 + (\Sigma \chi^\dagger)^2 \}$

## NLO Lagrangian

In [52]:
ls = [var("l{}".format(i), latex_name = "l_{}".format(i)) for i in range(1, 8)]
hs = [var("h{}".format(i), latex_name = "h_{}".format(i)) for i in range(1, 8)]

In [54]:
LNLO = (
    l1 / 4 * term_l1
    + l2 / 4 * term_l2
    + (l3 + h1 - h3) / 16 * term_l3
    + l4 / 8 * term_l4
    + (h1 - h3 - l4 - l7) / 16 * term_l7
    + (h1 + h3 - l4) / 4 * term_chisq
    - (h1 - h3 - l4) / 8 * term_last
)
print_e(LNLO)

1 :


-4*(l3*mm^2*sin(a)^2 + l4*mm^2*sin(a)^2 - dm^2*h3 - h1*mm^2 - l3*mm^2)*B0^2

2*B0*d^2*l4*mm*mu^2*cos(a)*sin(a)^2

(d^4*l1 + dmu^2*dnu^2*l2)*mu^4*sin(a)^4

e :


...

2*(4*B0*l3*mm^2*pi1(x)^2*sin(a)^2 + 4*B0*l4*mm^2*pi1(x)^2*sin(a)^2 + 2*B0*l3*mm^2*pi2(x)^2*sin(a)^2 + 2*B0*l4*mm^2*pi2(x)^2*sin(a)^2 + 2*B0*l3*mm^2*pi3(x)^2*sin(a)^2 + 2*B0*l4*mm^2*pi3(x)^2*sin(a)^2 - 2*B0*l3*mm^2*pi1(x)^2 - 2*B0*l4*mm^2*pi1(x)^2 - 2*B0*l3*mm^2*pi2(x)^2 - 2*B0*l4*mm^2*pi2(x)^2 + 2*B0*dm^2*l7*pi3(x)^2 - 2*B0*l3*mm^2*pi3(x)^2 - 2*B0*l4*mm^2*pi3(x)^2 + l4*mm*cos(a)*diff(pi1(x), x)^2 + l4*mm*cos(a)*diff(pi2(x), x)^2 + l4*mm*cos(a)*diff(pi3(x), x)^2)*B0

-4*(cos(a)^2*pi2(x)*diff(pi1(x), x) - 2*cos(a)^2*pi1(x)*diff(pi2(x), x) + pi1(x)*diff(pi2(x), x))*B0*d*l4*mm*mu

-(9*B0*d^2*l4*mm*cos(a)*pi1(x)^2*sin(a)^2 + 3*B0*d^2*l4*mm*cos(a)*pi2(x)^2*sin(a)^2 + 3*B0*d^2*l4*mm*cos(a)*pi3(x)^2*sin(a)^2 - 2*B0*d^2*l4*mm*cos(a)*pi1(x)^2 - 2*B0*d^2*l4*mm*cos(a)*pi2(x)^2 - dnu^2*l2*pimu2(x)^2*sin(a)^2 - 2*dmu*dnu*l2*pimu1(x)*pinu1(x)*sin(a)^2 - 4*dmu*dnu*l2*pimu2(x)*pinu2(x)*sin(a)^2 - dmu^2*l2*pinu2(x)^2*sin(a)^2 - 2*dmu*dnu*l2*pimu3(x)*pinu3(x)*sin(a)^2 - 2*d^2*l1*sin(a)^2*diff(pi1(x), x)^2 - 6*d^2*l1*sin(a)^2*diff(pi2(x), x)^2 - 2*d^2*l1*sin(a)^2*diff(pi3(x), x)^2)*mu^2

-2*(dmu*dnu^2*l2*pi2(x)*pimu1(x) - 3*dmu*dnu^2*l2*pi1(x)*pimu2(x) + dmu^2*dnu*l2*pi2(x)*pinu1(x) - 3*dmu^2*dnu*l2*pi1(x)*pinu2(x) + 2*d^3*l1*pi2(x)*diff(pi1(x), x) - 6*d^3*l1*pi1(x)*diff(pi2(x), x))*mu^3*cos(a)*sin(a)^2

-2*(d^4*l1 + dmu^2*dnu^2*l2)*(4*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - 3*pi1(x)^2 - pi2(x)^2)*mu^4*sin(a)^2

In [55]:
L_stat = LNLO.coefficients(e)[0][0].full_simplify().subs(dmu^2==1, dnu^2==1, d==1)
L_stat.full_simplify()

2*B0*l4*mm*mu^2*cos(a)*sin(a)^2 + 4*B0^2*dm^2*h3 + (l1*sin(a)^4 + l2*sin(a)^4)*mu^4 - 4*(B0^2*l4*sin(a)^2 - B0^2*h1 + (B0^2*sin(a)^2 - B0^2)*l3)*mm^2

In [56]:
eq = L_stat.diff(a).full_simplify() == 0
eq

-2*(3*B0*sin(a)^3 - 2*B0*sin(a))*l4*mm*mu^2 + 4*(l1*cos(a)*sin(a)^3 + l2*cos(a)*sin(a)^3)*mu^4 - 8*(B0^2*l3*cos(a)*sin(a) + B0^2*l4*cos(a)*sin(a))*mm^2 == 0

In [59]:
solve(eq, a)

[a == arccos(1/2*l4*mu^2/(B0*l3*mm + B0*l4*mm)), a == 0]

In [58]:
var("y")
L_lin = L.coefficients(e)[1][0].subs(p[0] == y).diff(y).subs(dmu^2==1, dnu^2==1, d==1).full_simplify()
L_lin.expand().full_simplify()

-2*(3*B0*sin(a)^3 - 2*B0*sin(a))*l4*mm*mu^2 + 4*(l1*cos(a)*sin(a)^3 + l2*cos(a)*sin(a)^3)*mu^4 - 8*(B0^2*l3*cos(a)*sin(a) + B0^2*l4*cos(a)*sin(a))*mm^2

In [48]:
L_stat.expand().diff(a).expand().full_simplify()

-2*(3*B0*sin(a)^3 - 2*B0*sin(a))*l4*mm*mu^2 + 4*(l1*cos(a)*sin(a)^3 + l2*cos(a)*sin(a)^3)*mu^4 - 8*(B0^2*l3*cos(a)*sin(a) + B0^2*l4*cos(a)*sin(a))*mm^2

In [50]:
F = 

SyntaxError: invalid syntax (1621629255.py, line 1)